In [ ]:
import os
import myServices 
import randForReg as rfr
import numpy as np
import pandas as pd
import sklearn
import joblib

In [ ]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Random Forest Section

In [ ]:
sklearn.metrics.get_scorer_names()

In [ ]:
DS = pd.read_csv('/Users/abdielfer/DESS/Internship2022/RNCanWork/datasetBasin1.csv')

In [ ]:
DS.isna().sum()

In [ ]:
DS.info()

In [ ]:
DS['percentage'].unique()

In [ ]:
DS['percentage'].value_counts()

In [ ]:
# DS['weights'] = DS['percentage'].map({0:3, 1: 60, 5: 37})
sample = DS.sample(n=10000) # or use : frac=0.05
print(sample.head)
sample['percentage'].unique()

In [ ]:
sample['percentage'] = sample['percentage']*10

In [ ]:
print(sample['percentage'].unique())

In [ ]:
sample['percentage'].value_counts()

In [ ]:
sample.drop(columns=['x_coord', 'y_coord'],inplace = True)
print(sample.head)

In [ ]:
sample.to_csv('./sample.csv')

In [ ]:
#Data import and exploration
datasetPath = './sample.csv'
rfReg  = rfr.implementRandomForestRegressor(datasetPath,'percentage', 0.2)
x_train,x_validation,y_train, y_validation = rfReg.getSplitedDataset()

In [ ]:
with timeit():
    bestEstimator = rfReg.fitRFRegressorWeighted(False)

In [3]:
# import executeRFReg as exe
with timeit():
    rfr.main()

 (8000, 7)   : (2000, 7)
Label balance on Training set:  
 0     7829
50     136
10      35
Name: percentage, dtype: int64
Label balance on Validation set:  
 0     1954
50      39
10       7
Name: percentage, dtype: int64
Fitting 3 folds for each of 8160 candidates, totalling 24480 fits
./models/rwReg/rfwgs_2207200845.pkl
      feature  importance
1    disToRiv       0.302
6   elevation       0.159
4        FAcc       0.138
5       slope       0.120
0  Unnamed: 0       0.108
2         TWI       0.093
3         TPI       0.080
The best parameters: {'bootstrap': True, 'max_depth': 12, 'max_features': 0.30000000000000004, 'max_leaf_nodes': None, 'n_estimators': 65}
Mean Absolute Error:  10.486235843642694
Mean Squared Error:  1014.1713592791683
Root Mean Squared Error:  31.846057201467943
runtime: 0:33:21.416096


ValueError: Found input variables with inconsistent numbers of samples: [2000, 2000, 8000]